In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

In [ ]:
test = pd.read_csv('include/test.csv')
train = pd.read_csv('include/train.csv').dropna()

In [ ]:
def substrings_in_string(big_string, substrings):
    for substring in substrings:
        if str(big_string).find(substring) != -1:
            return substring
    return ''

In [ ]:
train_reformated = train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
cabin_list = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'Unknown']
train_reformated['Deck'] = train['Cabin'].map(lambda x: substrings_in_string(x, cabin_list))
train_reformated['Family_Size'] = train_reformated['SibSp'] + train_reformated['Parch']
train_reformated['Fare_Per_Person'] = train_reformated['Fare'] /(train_reformated['Family_Size'] + 1)

train_reformated['Sex'] =  LabelEncoder().fit_transform(train_reformated['Sex'])
train_reformated['Embarked'] = LabelEncoder().fit_transform(train_reformated['Embarked'])
train_reformated['Deck'] = LabelEncoder().fit_transform(train_reformated['Deck'])
print(train_reformated.head())

In [ ]:
clf = RandomForestClassifier()
clf.fit(train_reformated.loc[:, train_reformated.columns != 'Survived'], train_reformated['Survived'])
print(clf.feature_importances_)

In [ ]:
test_passengerId = test['PassengerId']
test_reformated = test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
cabin_list = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'Unknown']
test_reformated['Deck'] = test['Cabin'].map(lambda x: substrings_in_string(x, cabin_list))
test_reformated['Family_Size'] = test_reformated['SibSp'] + test_reformated['Parch']
test_reformated['Fare_Per_Person'] = test_reformated['Fare'] /(test_reformated['Family_Size'] + 1)

test_reformated['Sex'] =  LabelEncoder().fit_transform(test_reformated['Sex'])
test_reformated['Embarked'] = LabelEncoder().fit_transform(test_reformated['Embarked'])
test_reformated['Deck'] = LabelEncoder().fit_transform(test_reformated['Deck'])

In [ ]:
print(test_reformated.columns[test_reformated.isnull().any()].tolist())
test_reformated = test_reformated.fillna(test_reformated.mean())

test_predictions = clf.predict(test_reformated).reshape(-1, 1)

combined = np.column_stack((test_passengerId, test_predictions))
df = pd.DataFrame(combined, columns=["PassengerId", "Survived"])

In [ ]:
df.to_csv('csv/answers_after_preprocess.csv',index=False)